<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Model Read

In [18]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import os
import glob
import seaborn as sns
sns.set()

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load tensorflow models
Load detection_pipe


In [19]:
path = BASE_FOLDER + './modeling/unsupervised/models_agg.dataframe'
df = pd.read_pickle(path)

In [20]:
mask1 = (df.model_type=='AutoEnc') & (df.feature_type=='MEL_den') & (df.feat_frames==5.0)
mask2 = (df.model_type=='IsoFor') & (df.feature_type=='MEL_den') & (df.feat_frames==3.0)
mask3 = (df.model_type=='IsoFor') & (df.feature_type=='PSD_raw')
mask4 = (df.model_type=='SVM') & (df.feature_type=='PSD_raw')
df = df[mask1 | mask2 | mask3 | mask4]
df = df.dropna(axis=1, thresh=1)


In [21]:
df = df.sort_values(by='roc_auc_aggr', ascending=False).drop_duplicates(subset=['machine', 'ID', 'SNR', 'model_type', 'feature_type'], keep='first')

In [22]:
for SNR in ['6dB', 'min6dB']:
    for machine in ['fan', 'pump', 'slider', 'valve']:
        for ID in ['00', '02']:#, '04', '06']:
            df_tmp = df[(df.machine==machine) & (df.SNR==SNR) & (df.ID==ID)]
            if df_tmp.shape[0] != 4:
                print(SNR, machine, ID, 'existing:', [(row.model_type, row.feature_type) for _, row in df_tmp.iterrows()])

6dB fan 00 existing: [('IsoFor', 'MEL_den'), ('AutoEnc', 'MEL_den'), ('IsoFor', 'PSD_raw')]
6dB fan 02 existing: [('IsoFor', 'PSD_raw'), ('AutoEnc', 'MEL_den'), ('IsoFor', 'MEL_den')]
6dB pump 00 existing: [('AutoEnc', 'MEL_den'), ('IsoFor', 'MEL_den'), ('IsoFor', 'PSD_raw')]
6dB pump 02 existing: [('IsoFor', 'MEL_den'), ('AutoEnc', 'MEL_den'), ('IsoFor', 'PSD_raw')]
6dB slider 00 existing: [('IsoFor', 'PSD_raw'), ('IsoFor', 'MEL_den'), ('AutoEnc', 'MEL_den')]
6dB slider 02 existing: [('IsoFor', 'PSD_raw'), ('AutoEnc', 'MEL_den'), ('IsoFor', 'MEL_den')]
6dB valve 00 existing: [('AutoEnc', 'MEL_den'), ('IsoFor', 'MEL_den'), ('IsoFor', 'PSD_raw')]
6dB valve 02 existing: [('IsoFor', 'PSD_raw'), ('AutoEnc', 'MEL_den'), ('IsoFor', 'MEL_den')]


In [23]:
ensemble_pipes_selection = {}
for SNR in ['6dB', 'min6dB']:
    for machine in ['fan', 'pump', 'slider', 'valve']:
        for ID in ['00', '02']:
            ensemble_pipes_selection[SNR+machine+ID] = []
            df_tmp = df[(df.machine==machine) & (df.SNR==SNR) & (df.ID==ID)]
            for _, row in df_tmp.iterrows():
                with open(row.model_path, 'rb') as f:
                    ensemble_pipes_selection[SNR+machine+ID].append(pickle.load(f))


C:\Users\bendi\.conda\envs\mimii-tf2-gpu\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [24]:
for ensemble_key, ensemble in ensemble_pipes_selection.items():
    for i, pipe in enumerate(ensemble):
        if not glob.glob(pipe.filepath[0:60] + '*'):
            old_path = pipe.filepath
            pipe.model = pipe._mdl(**pipe.model_args)
            #print(pipe.task)
            try:
                pipe.task['path_descr'] = glob.glob(BASE_FOLDER 
                                #+ '/dataset/extdia_v1*/{}{}{}_EDiaV1*aug'.format(pipe.task['machine'], pipe.task['SNR'], pipe.task['ID'])
                                + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(pipe.task['machine'], pipe.task['SNR'], pipe.task['ID'])
                                + "*pandaDisc*.pkl", recursive=True)[0]
            except:
                pipe.task['path_descr'] = glob.glob(BASE_FOLDER 
                                + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(pipe.task['machine'], pipe.task['SNR'], pipe.task['ID'])
                                + "*pandaDisc*.pkl", recursive=True)[0]
            try:
                pipe.run_pipe(pipe.task)
            except:
                print(pipe.task)

../..//dataset\extdia_v1\fanmin6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
{'path_descr': '../..//dataset\\extdia_v1\\fanmin6dB00_EDiaV1HP_pandaDisc.pkl', 'feat': {'function': 'flat'}, 'feat_col': 'PSD_raw', 'SNR': 'min6dB', 'machine': 'fan', 'ID': '00', 'BASE_FOLDER': '../../'}
../..//dataset\extdia_v1\fanmin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
{'path_descr': '../..//dataset\\extdia_v1\\fanmin6dB02_EDiaV1HP_pandaDisc.pkl', 'feat': {'function': 'flat'}, 'feat_col': 'PSD_raw', 'SNR': 'min6dB', 'machine': 'fan', 'ID': '02', 'BASE_FOLDER': '../../'}
../..//dataset\extdia_v1\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
0.9798501280262739
model fitted successful

In [25]:
ensemble_pipes = {}
paths = {}
for SNR in ['6dB', 'min6dB']:
    for machine in ['fan', 'pump', 'slider', 'valve']:
        for ID in ['00', '02']:
            paths[SNR+machine+ID] = glob.glob(BASE_FOLDER + 'modeling/ensemble/pipes/*_{}_{}_ID{}*.pkl'.format(SNR,machine,ID))
            ensemble_pipes[SNR+machine+ID] = []
            for path in paths[SNR+machine+ID]:
                with open(path, 'rb') as f:
                    ensemble_pipes[SNR+machine+ID].append(pickle.load(f))



C:\Users\bendi\.conda\envs\mimii-tf2-gpu\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [26]:
ensemble_dfs = {}
for ensemble_key, ensemble in ensemble_pipes.items():
    for i, pipe in enumerate(ensemble):

        if i==0:
            ensemble_dfs[ensemble_key] = pd.DataFrame(pipe.df_test.groupby(by='path').sum().pred_scores.rename(pipe.model.name+'_'+pipe.task['feat_col']+'_pred_scores').reindex())
            ground_truth = pipe.df_test.groupby(by='path').median().abnormal.rename('ground_truth')
            score_unweighted = pd.Series(np.zeros(ensemble_dfs[ensemble_key].iloc[:,0].shape), index=ensemble_dfs[ensemble_key].index)
            score_weighted = score_unweighted
        else:
            ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(pipe.df_test.groupby(by='path').sum().pred_scores.rename(pipe.model.name+'_'+pipe.task['feat_col']+'_pred_scores'))
        
        if pipe.model.name == 'IsoFor':
            if pipe.task['feat_col'] == 'PSD_raw':
                weight = 1.
            else:
                weight = .75
        elif pipe.model.name == 'AutoEnc':
            weight = 1.25
        elif pipe.model.name == 'SVM':
            weight = 1.
        score_weighted = score_weighted + weight * (ensemble_dfs[ensemble_key].iloc[:,-1] - ensemble_dfs[ensemble_key].iloc[:,-1].mean())/ensemble_dfs[ensemble_key].iloc[:,-1].std()
        score_unweighted = score_unweighted + (ensemble_dfs[ensemble_key].iloc[:,-1] - ensemble_dfs[ensemble_key].iloc[:,-1].mean())/ensemble_dfs[ensemble_key].iloc[:,-1].std()
    
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(score_weighted.rename('score_agg_weighted'))
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(score_unweighted.rename('score_agg_unweighted'))
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(ground_truth.replace(to_replace=1, value=-1).replace(to_replace=0, value=1))

ValueError: columns overlap but no suffix specified: Index(['SVM_PSD_raw_pred_scores'], dtype='object')

In [ ]:
results = pd.DataFrame([])
for ensemble_key, ensemble in ensemble_dfs.items():
    data = [roc_auc_score(ensemble_dfs[ensemble_key].ground_truth, ensemble_dfs[ensemble_key].score_agg_weighted), 
            roc_auc_score(ensemble_dfs[ensemble_key].ground_truth, ensemble_dfs[ensemble_key].score_agg_unweighted)]

    cols = ['score_aggr_weighted', 'score_aggr_unweighted']
    for pipe in ensemble_pipes[ensemble_key]:
        data.append(pipe.roc_auc)
        cols.append('score_'+pipe.model.name+pipe.task['feat_col'])
    results = results.append(pd.DataFrame(data=[data], columns=cols, index=[ensemble_key]))

In [ ]:
results

In [ ]:
results.score_aggr_weighted.mean()

In [ ]:
results.score_aggr_unweighted.mean()

In [ ]:
results.score_AutoEncMEL_den.mean()

In [ ]:
results.score_IsoForMEL_den.mean()

In [ ]:
results.score_IsoForPSD_raw.mean()

In [ ]:
for col in predictions['6dBpump00'].columns:
    s_tmp = s_tmp + (predictions['min6dBpump00'][col] - predictions['min6dBpump00'][col].mean())/predictions['min6dBpump00'][col].std()
predictions['min6dBpump00'].join(s_tmp.rename('score_agg'))

In [ ]:
s_tmp